# 2022-48 Census Building Permit ETL QC

## Library Imports

In [1]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

## Downloading Raw Data

## Downloading Staging Data